# Newyork vs Paris vs London

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim 
import geocoder
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import unicodedata

## Newyork

In [2]:
ny = pd.read_html('https://en.wikipedia.org/wiki/Neighborhoods_in_New_York_City')
newyork = ny[0]
newyork.head()

,Community Board(CB),Areakm2,Pop.Census2010,Pop./km2,Neighborhoods
0,Bronx CB 1,7.17,91497,12761,"Melrose, Mott Haven, Port Morris"
1,Bronx CB 2,5.54,52246,9792,"Hunts Point, Longwood"
2,Bronx CB 3,4.07,79762,19598,"Claremont, Concourse Village, Crotona Park, Mo..."
3,Bronx CB 4,5.28,146441,27735,"Concourse, Highbridge"
4,Bronx CB 5,3.55,128200,36145,"Fordham, Morris Heights, Mount Hope, Universit..."


In [3]:
newyork.rename(columns={'Community Board(CB)': 'Boroughs'}, inplace=True)
newyork.dropna(inplace=True)

In [4]:
newyork.head()

,Boroughs,Areakm2,Pop.Census2010,Pop./km2,Neighborhoods
0,Bronx CB 1,7.17,91497,12761,"Melrose, Mott Haven, Port Morris"
1,Bronx CB 2,5.54,52246,9792,"Hunts Point, Longwood"
2,Bronx CB 3,4.07,79762,19598,"Claremont, Concourse Village, Crotona Park, Mo..."
3,Bronx CB 4,5.28,146441,27735,"Concourse, Highbridge"
4,Bronx CB 5,3.55,128200,36145,"Fordham, Morris Heights, Mount Hope, Universit..."


In [5]:
unicode = []
for b in newyork['Boroughs']:
    unicode.append(unicodedata.normalize("NFKD", b))
    
newyork['Boroughs'] = unicode

In [6]:
newyork['Boroughs'] = newyork['Boroughs'].str.split('CB')

boroughs = []
for item in newyork['Boroughs']:
    boroughs.append(item[0])
    
newyork['Boroughs'] = boroughs

newyork.drop(['Areakm2','Pop.Census2010', 'Pop./km2', 'Neighborhoods'], axis=1, inplace=True)
newyork = newyork.drop_duplicates(subset=['Boroughs'])

In [7]:
newyork.head()

,Boroughs
0,Bronx
12,Brooklyn
30,Manhattan
42,Queens
56,Staten Island


Get the coordinates of all the boroughs using geocoder

In [8]:
def lat_long(borough, city):
    geo_code = geocoder.arcgis('{} {}'.format(borough, city))
    coords = geo_code.latlng
    lat = coords[0]
    long = coords[1]
    return lat, long

In [9]:
latitude_list =[]
longtitude_list = []

for index, row in newyork.iterrows():
    boroughs = row['Boroughs']
    
    latitude, longtitude = lat_long(boroughs, 'New York, US')
    latitude_list.append(latitude)
    longtitude_list.append(longtitude)

In [10]:
newyork['Latitude'] = latitude_list
newyork['Longitude'] = longtitude_list

newyork.head()

C:\Users\Babatolatemi\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Babatolatemi\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Boroughs,Latitude,Longitude
0,Bronx,40.82600,-73.92309
12,Brooklyn,40.69245,-73.99036
30,Manhattan,40.71450,-74.00601
42,Queens,40.71400,-73.82999
56,Staten Island,40.64242,-74.07527


In [11]:
newyork.shape

(5, 3)

In [12]:
newyork['Boroughs'].nunique()

5

Get the coordinates of New York City

In [13]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
ny_latitude = location.latitude
ny_longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(ny_latitude, ny_longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


## Paris

In [14]:
p = pd.read_html('https://en.wikipedia.org/wiki/Arrondissements_of_Paris')

In [15]:
p[2]

,"Arrondissement (R for Right Bank, L for Left Bank)",Name,Area (km2),Population(2017 estimate),Density (2017)(inhabitants per km2),Peak of population,Mayor,2020-2026
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,"Louvre, Bourse, Temple, Hôtel-de-Ville",5.59 km2 (2.16 sq mi),100196,17924,before 1861,Ariel Weil (PS),NaN
1,5th (Ve) L,Panthéon,2.541 km2 (0.981 sq mi),59631,23477,1911,Florence Berthout (DVD),NaN
2,6th (VIe) L,Luxembourg,2.154 km2 (0.832 sq mi),41976,19524,1911,Jean-Pierre Lecoq (LR),NaN
3,7th (VIIe) L,Palais-Bourbon,4.088 km2 (1.578 sq mi),52193,12761,1926,Rachida Dati (LR),NaN
4,8th (VIIIe) R,Élysée,3.881 km2 (1.498 sq mi),37368,9631,1891,Jeanne d'Hauteserre (LR),NaN
5,9th (IXe) R,Opéra,2.179 km2 (0.841 sq mi),60071,27556,1901,Delphine Bürkli (DVD),NaN
6,10th (Xe) R,Entrepôt,2.892 km2 (1.117 sq mi),90836,31431,1881,Alexandra Cordebard (PS),NaN
7,11th (XIe) R,Popincourt,3.666 km2 (1.415 sq mi),147470,40183,1911,François Vauglin (PS),NaN
8,12th (XIIe) R,Reuilly,16.324 km2 (6.303 sq mi)¹6.377 km2 (2.462 sq mi)²,141287,"8,657¹21,729²",1962,Emmanuelle Pierre-Marie (The Greens),NaN
9,13th (XIIIe) L,Gobelins,7.146 km2 (2.759 sq mi),183399,25650,20055,Jérôme Coumet (PS),NaN


In [16]:
paris = p[2][['Arrondissement (R for Right Bank, L for Left Bank)', 'Name']]
paris.head()

,"Arrondissement (R for Right Bank, L for Left Bank)",Name
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,"Louvre, Bourse, Temple, Hôtel-de-Ville"
1,5th (Ve) L,Panthéon
2,6th (VIe) L,Luxembourg
3,7th (VIIe) L,Palais-Bourbon
4,8th (VIIIe) R,Élysée


In [17]:
paris.shape

(17, 2)

In [18]:
paris['Name'] = paris['Name'].str.split(',')

# convert list of pd.Series then stack it
paris = (paris.set_index(['Arrondissement (R for Right Bank, L for Left Bank)'])['Name'].apply(pd.Series).stack()
         .reset_index().drop('level_1', axis=1).rename(
             columns={0:'District', 'Arrondissement (R for Right Bank, L for Left Bank)':'Arrondissement'}))
paris.head()

C:\Users\Babatolatemi\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Arrondissement,District
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,Louvre
1,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,Bourse
2,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,Temple
3,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,Hôtel-de-Ville
4,5th (Ve) L,Panthéon


In [19]:
paris.shape

(20, 2)

In [20]:
paris['District'].nunique()

20

In [21]:
latitude_list =[]
longtitude_list = []

for index, row in paris.iterrows():
    district = row['District']
    
    latitude, longtitude = lat_long(district, 'Paris, France')
    latitude_list.append(latitude)
    longtitude_list.append(longtitude)

In [22]:
paris['Latitude'] = latitude_list
paris['Longitude'] = longtitude_list

paris.head()

,Arrondissement,District,Latitude,Longitude
0,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,Louvre,48.861020,2.335870
1,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,Bourse,48.869100,2.340220
2,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,Temple,48.864660,2.364260
3,Paris Centre 1st (Ier) / 2nd (IIe) / 3rd (IIIe...,Hôtel-de-Ville,48.857101,2.353064
4,5th (Ve) L,Panthéon,48.846210,2.346110


In [23]:
paris.shape

(20, 4)

In [24]:
address = 'Paris, France'

geolocator = Nominatim(user_agent="pa_explorer")
location = geolocator.geocode(address)
pa_latitude = location.latitude
pa_longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(pa_latitude, pa_longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


## London

In [25]:
ln = pd.read_html('https://en.wikipedia.org/wiki/List_of_places_in_London#Neighbourhoods')
london = ln[0][['London borough', 'Sub-region[1]']]

In [26]:
london.head()

,London borough,Sub-region[1]
0,Barking and Dagenham,North East
1,Barnet,North
2,Bexley,South East
3,Brent,West
4,Bromley,South East


In [27]:
london.shape

(32, 2)

In [28]:
london['London borough'].nunique()

32

In [29]:
latitude_list =[]
longtitude_list = []

for index, row in london.iterrows():
    borough = row['London borough']
    
    latitude, longtitude = lat_long(borough, 'London, UK')
    latitude_list.append(latitude)
    longtitude_list.append(longtitude)

In [30]:
london['Latitude'] = latitude_list
london['Longitude'] = longtitude_list

london.head()

C:\Users\Babatolatemi\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Babatolatemi\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,London borough,Sub-region[1],Latitude,Longitude
0,Barking and Dagenham,North East,51.543932,0.133157
1,Barnet,North,51.627300,-0.253760
2,Bexley,South East,51.452078,0.069931
3,Brent,West,51.475773,0.067826
4,Bromley,South East,51.601511,-0.066365


In [31]:
london.shape

(32, 4)

In [32]:
address = 'London, UK'

geolocator = Nominatim(user_agent="lo_explorer")
location = geolocator.geocode(address)
lo_latitude = location.latitude
lo_longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(lo_latitude, lo_longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


# Exploring the venues in these cities

The *getNearbyVenues* function extracts the venues in the specified location and returns a dataframe containing all the information.

NOTE: The function has a new column naming system, the Locations (Disticts, London Borough, Borough) are all going to be called 'Borough' in the resulting dataframe

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    """ INPUT:
            names: the names of the locations where we want to extract venues using Foursquare API
            latitudes: location latitudes
            longitudes: location longitudes
            radius: foursquare search radius
            
        OUTPUT:
            nearby_venues: A dataframe of all the venues in the different locations extracted from foursquare.
        """
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
CLIENT_ID = 'NOBZYQP2PWG3LPG0DDHDGS1STD1OW1PL5LG01X2DZPCJM414' # your Foursquare ID
CLIENT_SECRET = 'PKIYHSTKE4QBUVWMZAFFYDTGIMBAQXODZJTYA3NRL5KGM0C0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NOBZYQP2PWG3LPG0DDHDGS1STD1OW1PL5LG01X2DZPCJM414
CLIENT_SECRET:PKIYHSTKE4QBUVWMZAFFYDTGIMBAQXODZJTYA3NRL5KGM0C0


In [35]:
newyork_venues = getNearbyVenues(names=newyork['Boroughs'],
                                   latitudes=newyork['Latitude'],
                                   longitudes=newyork['Longitude']
                                  )

Bronx 
Brooklyn 
Manhattan 
Queens 
Staten Island 


In [36]:
newyork_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,40.826,-73.92309,Bronx Drafthouse,40.827998,-73.924677,Beer Bar
1,Bronx,40.826,-73.92309,La Bodega,40.826949,-73.925171,Deli / Bodega
2,Bronx,40.826,-73.92309,Blink Fitness,40.825185,-73.921825,Gym
3,Bronx,40.826,-73.92309,Bald Vinny's House of Tees,40.827359,-73.926192,Clothing Store
4,Bronx,40.826,-73.92309,Gerard Discount Liquor,40.827836,-73.924729,Liquor Store


In [37]:
paris_venues = getNearbyVenues(names=paris['District'],
                                   latitudes=paris['Latitude'],
                                   longitudes=paris['Longitude']
                                  )

Louvre
 Bourse
 Temple
 Hôtel-de-Ville
Panthéon
Luxembourg
Palais-Bourbon
Élysée
Opéra
Entrepôt
Popincourt
Reuilly
Gobelins
Observatoire
Vaugirard
Passy
Batignolles-Monceau
Butte-Montmartre
Buttes-Chaumont
Ménilmontant


In [38]:
paris_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Louvre,48.86102,2.33587,Musée du Louvre,48.860847,2.336440,Art Museum
1,Louvre,48.86102,2.33587,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
2,Louvre,48.86102,2.33587,Vestige de la Forteresse du Louvre,48.861577,2.333508,Historic Site
3,Louvre,48.86102,2.33587,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
4,Louvre,48.86102,2.33587,Cour Napoléon,48.861172,2.335088,Plaza


In [39]:
london_venues = getNearbyVenues(names=london['London borough'],
                                   latitudes=london['Latitude'],
                                   longitudes=london['Longitude']
                                  )

Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


In [40]:
london_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham,51.543932,0.133157,Haz Wine and Dine,51.540833,0.137116,Turkish Restaurant
1,Barking and Dagenham,51.543932,0.133157,Modest Touch Clothing,51.539952,0.132989,Women's Store
2,Barnet,51.627300,-0.253760,Mill Hill Golf Club,51.627944,-0.253174,Golf Course
3,Bexley,51.452078,0.069931,Eltham Park South,51.455424,0.066433,Park
4,Bexley,51.452078,0.069931,"Green Chain Walk, Eltham",51.449917,0.067186,Trail


## Analyze Each Borough

**NEWYORK**

In [41]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Borough'] = newyork_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_onehot.head()

,Borough,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Beer Bar,Bistro,Boat or Ferry,Bookstore,Boxing Gym,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Diner,Discount Store,Donut Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Laundry Service,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Museum,Music Store,Nail Salon,New American Restaurant,Office,Optical Shop,Outlet Mall,Park,Pharmacy,Pizza Place,Playground,Plaza,Pub,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Theme Restaurant,Tour Provider,Toy / Game Store,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
newyork_onehot.shape

(361, 139)

**PARIS**

In [43]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Borough'] = paris_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Borough,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Brasserie,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soup Place,Southwestern French Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Trattoria/Osteria,Travel Agency,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Louvre,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Louvre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Louvre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [44]:
paris_onehot.shape

(1405, 202)

**LONDON**

In [45]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Borough'] = london_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Borough,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,Bagel Shop,Bakery,Bar,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Dry Cleaner,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Escape Room,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Gelato Shop,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Himalayan Restaurant,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Restaurant,Rock Club,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Barnet,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0

In [46]:
london_onehot.shape

(1055, 204)

In [47]:
newyork_onehot['City'] = 'Newyork'
paris_onehot['City'] = 'Paris'
london_onehot['City'] = 'London'

In [48]:
full = pd.concat([newyork_onehot, paris_onehot, london_onehot])
full.head()

,Borough,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Beer Bar,Bistro,Boat or Ferry,Bookstore,Boxing Gym,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Diner,Discount Store,Donut Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Laundry Service,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Museum,Music Store,Nail Salon,New American Restaurant,Office,Optical Shop,Outlet Mall,Park,Pharmacy,Pizza Place,Playground,Plaza,Pub,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Theme Restaurant,Tour Provider,Toy / Game Store,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Wine Shop,Wings Joint,Women's Store,Yoga Studio,City,Afghan Restaurant,African Restaurant,Alsatian Restaurant,Arcade,Art Museum,Auvergne Restaurant,BBQ Joint,Basque Restaurant,Bed & Breakfast,Beer Garden,Beer Store,Bike Rental / Bike Share,Board Shop,Boarding House,Boutique,Brasserie,Breakfast Spot,Bridge,Bus Station,Bus Stop,Butcher,Casino,Cheese Shop,Church,Circus,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Corsican Restaurant,Creperie,Cultural Center,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Exhibit,Fish & Chips Shop,Food & Drink Shop,Fountain,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gluten-free Restaurant,Government Building,Health Food Store,Hookah Bar,Hostel,Indie Movie Theater,Israeli Restaurant,Jewish Restaurant,Kids Store,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Memorial Site,Miscellaneous Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Music Venue,Nightclub,Noodle House,Okonomiyaki Restaurant,Organic Grocery,Outdoor Sculpture,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Planetarium,Platform,Poke Place,Pool,Portuguese Restaurant,Ramen Restaurant,Record Shop,Resort,Russian Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Shoe Store,Soup Place,Southwestern French Restaurant,Souvenir Shop,Street Art,Tailor Shop,Taiwanese Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Tibetan Restaurant,Trail,Trattoria/Osteria,Travel Agency,Turkish Restaurant,Udon Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Wine Bar,Arepa Restaurant,Argentinian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,Bike Shop,Botanical Garden,Bulgarian Restaurant,Canal,Canal Lock,Candy Store,Caucasian Restaurant,Climbing Gym,Distillery,Dive Bar,Dry Cleaner,Eastern European Restaurant,Empanada Restaurant,Event Space,Film Studio,Fish Market,Flea Market,Food,Food Stand,Fruit & Vegetable Stor

In [49]:
full = full.fillna(0)

**Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [50]:
cities_grouped = full.groupby('Borough').mean().reset_index()
cities_grouped.head()

,Borough,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Beer Bar,Bistro,Boat or Ferry,Bookstore,Boxing Gym,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Diner,Discount Store,Donut Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Laundry Service,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Museum,Music Store,Nail Salon,New American Restaurant,Office,Optical Shop,Outlet Mall,Park,Pharmacy,Pizza Place,Playground,Plaza,Pub,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Theme Restaurant,Tour Provider,Toy / Game Store,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Afghan Restaurant,African Restaurant,Alsatian Restaurant,Arcade,Art Museum,Auvergne Restaurant,BBQ Joint,Basque Restaurant,Bed & Breakfast,Beer Garden,Beer Store,Bike Rental / Bike Share,Board Shop,Boarding House,Boutique,Brasserie,Breakfast Spot,Bridge,Bus Station,Bus Stop,Butcher,Casino,Cheese Shop,Church,Circus,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Corsican Restaurant,Creperie,Cultural Center,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Exhibit,Fish & Chips Shop,Food & Drink Shop,Fountain,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gluten-free Restaurant,Government Building,Health Food Store,Hookah Bar,Hostel,Indie Movie Theater,Israeli Restaurant,Jewish Restaurant,Kids Store,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Memorial Site,Miscellaneous Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Music Venue,Nightclub,Noodle House,Okonomiyaki Restaurant,Organic Grocery,Outdoor Sculpture,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Planetarium,Platform,Poke Place,Pool,Portuguese Restaurant,Ramen Restaurant,Record Shop,Resort,Russian Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Shoe Store,Soup Place,Southwestern French Restaurant,Souvenir Shop,Street Art,Tailor Shop,Taiwanese Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Tibetan Restaurant,Trail,Trattoria/Osteria,Travel Agency,Turkish Restaurant,Udon Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Wine Bar,Arepa Restaurant,Argentinian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,Bike Shop,Botanical Garden,Bulgarian Restaurant,Canal,Canal Lock,Candy Store,Caucasian Restaurant,Climbing Gym,Distillery,Dive Bar,Dry Cleaner,Eastern European Restaurant,Empanada Restaurant,Event Space,Film Studio,Fish Market,Flea Market,Food,Food Stand,Fruit & Vegetable Store,Gar

In [51]:
num_top_venues = 5

for hood in cities_grouped['Borough']:
    print("----"+hood+"----")
    temp = cities_grouped[cities_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Bourse----
                 venue  freq
0    French Restaurant  0.09
1  Japanese Restaurant  0.07
2             Wine Bar  0.04
3                Hotel  0.04
4               Bistro  0.04


---- Hôtel-de-Ville----
               venue  freq
0  French Restaurant  0.10
1     Ice Cream Shop  0.06
2        Coffee Shop  0.04
3     Clothing Store  0.03
4        Art Gallery  0.03


---- Temple----
                venue  freq
0   French Restaurant  0.09
1            Wine Bar  0.05
2               Hotel  0.04
3      Sandwich Place  0.03
4  Italian Restaurant  0.03


----Barking and Dagenham----
                   venue  freq
0          Women's Store   0.5
1     Turkish Restaurant   0.5
2  Performing Arts Venue   0.0
3            Pastry Shop   0.0
4      Outdoor Sculpture   0.0


----Barnet----
                    venue  freq
0             Golf Course   1.0
1     American Restaurant   0.0
2     Moroccan Restaurant   0.0
3         Organic Grocery   0.0
4  Okonomiyaki Restaurant   0.0


----Bati

4             Bistro  0.04


----Richmond upon Thames----
                    venue  freq
0                    Lake  0.67
1         Nature Preserve  0.33
2     American Restaurant  0.00
3       Outdoor Sculpture  0.00
4  Okonomiyaki Restaurant  0.00


----Southwark----
                venue  freq
0         Coffee Shop  0.10
1                 Pub  0.09
2  Seafood Restaurant  0.04
3            Wine Bar  0.04
4  Italian Restaurant  0.04


----Staten Island ----
                 venue  freq
0       Clothing Store  0.11
1   Italian Restaurant  0.06
2                  Bar  0.06
3  Sporting Goods Shop  0.04
4        Deli / Bodega  0.04


----Sutton----
                venue  freq
0          Food Truck   0.1
1                Park   0.1
2   Indian Restaurant   0.1
3  Chinese Restaurant   0.1
4          Playground   0.1


----Tower Hamlets----
                  venue  freq
0                   Pub  0.15
1    Chinese Restaurant  0.08
2           Supermarket  0.08
3                Bakery  0.08
4  F

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted[['Borough']] = cities_grouped[['Borough']]

for ind in np.arange(cities_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cities_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bourse,French Restaurant,Japanese Restaurant,Wine Bar,Bistro,Hotel,Salad Place,Korean Restaurant,Furniture / Home Store,Cocktail Bar,Burger Joint
1,Hôtel-de-Ville,French Restaurant,Ice Cream Shop,Coffee Shop,Clothing Store,Gourmet Shop,Bar,Plaza,Italian Restaurant,Art Gallery,Tea Room
2,Temple,French Restaurant,Wine Bar,Hotel,Sandwich Place,Bistro,Clothing Store,Art Gallery,Italian Restaurant,Cocktail Bar,Speakeasy
3,Barking and Dagenham,Women's Store,Turkish Restaurant,Tree,Plaza,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop
4,Barnet,Golf Course,Tree,Seafood Restaurant,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden


In [54]:
# set number of clusters
kclusters = 10

cities_grouped_clustering = cities_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cities_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 7, 1, 0, 9, 3, 8, 8, 8, 0, 0, 5, 6, 5, 5, 0, 0, 5, 5, 8,
       8, 5, 8, 8, 9, 5, 8, 5, 5, 8, 0, 0, 5, 8, 0, 4, 0, 0, 0, 0, 0, 8,
       8, 0, 2, 5, 8, 8, 5, 0, 5, 5, 5, 0])

In [55]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cities_merged = full[['Borough', 'City']]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cities_merged = cities_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

cities_merged.head()

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
1,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
2,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
3,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
4,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store


#### Cluster 1

In [56]:
cities_merged.loc[cities_merged['Cluster Labels'] == 0]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store
1,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store
2,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store
3,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store
4,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store
5,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store
6,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store
7,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store
8,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store
9,Louvre,Paris,0,Hotel,Plaza,French Restaurant,Café,Italian Restaurant,Art Museum,Coffee Shop,Cosmetics Shop,Garden,Clothing Store


#### Cluster 2

In [57]:
cities_merged.loc[cities_merged['Cluster Labels'] == 1]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Barnet,London,1,Golf Course,Tree,Seafood Restaurant,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden


#### Cluster 3

In [58]:
cities_merged.loc[cities_merged['Cluster Labels'] == 2]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
835,Richmond upon Thames,London,2,Lake,Nature Preserve,Pub,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place
836,Richmond upon Thames,London,2,Lake,Nature Preserve,Pub,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place
837,Richmond upon Thames,London,2,Lake,Nature Preserve,Pub,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place


#### Cluster 4

In [59]:
cities_merged.loc[cities_merged['Cluster Labels'] == 3]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Brent,London,3,Middle Eastern Restaurant,Home Service,Convenience Store,Tree,Shipping Store,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden
10,Brent,London,3,Middle Eastern Restaurant,Home Service,Convenience Store,Tree,Shipping Store,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden
11,Brent,London,3,Middle Eastern Restaurant,Home Service,Convenience Store,Tree,Shipping Store,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden


#### Cluster 5 

In [60]:
cities_merged.loc[cities_merged['Cluster Labels'] == 4]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
816,Newham,London,4,Bus Stop,Soccer Field,Supermarket,Coffee Shop,Tree,Seafood Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop
817,Newham,London,4,Bus Stop,Soccer Field,Supermarket,Coffee Shop,Tree,Seafood Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop
818,Newham,London,4,Bus Stop,Soccer Field,Supermarket,Coffee Shop,Tree,Seafood Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop
819,Newham,London,4,Bus Stop,Soccer Field,Supermarket,Coffee Shop,Tree,Seafood Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop


#### Cluster 6

In [61]:
cities_merged.loc[cities_merged['Cluster Labels'] == 5]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
159,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa
160,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa
161,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa
162,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa
163,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa
164,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa
165,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa
166,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa
167,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa
168,Manhattan,Newyork,5,Coffee Shop,Sandwich Place,Gym / Fitness Center,American Restaurant,Hotel,Cocktail Bar,Yoga Studio,Bakery,Italian Restaurant,Spa


#### Cluster 7

In [62]:
cities_merged.loc[cities_merged['Cluster Labels'] == 6]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,Croydon,London,6,Fast Food Restaurant,Café,Playground,Park,Chinese Restaurant,Tree,Seafood Restaurant,Salad Place,Salon / Barbershop,Sandwich Place
98,Croydon,London,6,Fast Food Restaurant,Café,Playground,Park,Chinese Restaurant,Tree,Seafood Restaurant,Salad Place,Salon / Barbershop,Sandwich Place
99,Croydon,London,6,Fast Food Restaurant,Café,Playground,Park,Chinese Restaurant,Tree,Seafood Restaurant,Salad Place,Salon / Barbershop,Sandwich Place
100,Croydon,London,6,Fast Food Restaurant,Café,Playground,Park,Chinese Restaurant,Tree,Seafood Restaurant,Salad Place,Salon / Barbershop,Sandwich Place
101,Croydon,London,6,Fast Food Restaurant,Café,Playground,Park,Chinese Restaurant,Tree,Seafood Restaurant,Salad Place,Salon / Barbershop,Sandwich Place


#### Cluster 8

In [63]:
cities_merged.loc[cities_merged['Cluster Labels'] == 7]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,London,7,Women's Store,Turkish Restaurant,Tree,Plaza,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop
1,Barking and Dagenham,London,7,Women's Store,Turkish Restaurant,Tree,Plaza,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop


#### Cluster 9

In [64]:
cities_merged.loc[cities_merged['Cluster Labels'] == 8]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
1,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
2,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
3,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
4,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
5,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
6,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
7,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
8,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store
9,Bronx,Newyork,8,Deli / Bodega,Sandwich Place,Fast Food Restaurant,Baseball Stadium,Sports Bar,Mexican Restaurant,Plaza,Pizza Place,Park,Clothing Store


#### Cluster 10

In [65]:
cities_merged.loc[cities_merged['Cluster Labels'] == 9]

,Borough,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Bexley,London,9,Park,Trail,Bus Stop,Indian Restaurant,Botanical Garden,Tree,Restaurant,Rock Club,Roof Deck,Salad Place
4,Bexley,London,9,Park,Trail,Bus Stop,Indian Restaurant,Botanical Garden,Tree,Restaurant,Rock Club,Roof Deck,Salad Place
5,Bexley,London,9,Park,Trail,Bus Stop,Indian Restaurant,Botanical Garden,Tree,Restaurant,Rock Club,Roof Deck,Salad Place
6,Bexley,London,9,Park,Trail,Bus Stop,Indian Restaurant,Botanical Garden,Tree,Restaurant,Rock Club,Roof Deck,Salad Place
7,Bexley,London,9,Park,Trail,Bus Stop,Indian Restaurant,Botanical Garden,Tree,Restaurant,Rock Club,Roof Deck,Salad Place
8,Bexley,London,9,Park,Trail,Bus Stop,Indian Restaurant,Botanical Garden,Tree,Restaurant,Rock Club,Roof Deck,Salad Place
584,Hounslow,London,9,Indian Restaurant,Fast Food Restaurant,Bus Stop,Grocery Store,Photography Studio,Convenience Store,Chinese Restaurant,Dessert Shop,Metro Station,Portuguese Restaurant
585,Hounslow,London,9,Indian Restaurant,Fast Food Restaurant,Bus Stop,Grocery Store,Photography Studio,Convenience Store,Chinese Restaurant,Dessert Shop,Metro Station,Portuguese Restaurant
586,Hounslow,London,9,Indian Restaurant,Fast Food Restaurant,Bus Stop,Grocery Store,Photography Studio,Convenience Store,Chinese Restaurant,Dessert Shop,Metro Station,Portuguese Restaurant
587,Hounslow,London,9,Indian Restaurant,Fast Food Restaurant,Bus Stop,Grocery Store,Photography Studio,Convenience Store,Chinese Restaurant,Dessert Shop,Metro Station,Portuguese Restaurant


### Conclusion:

Something interesting happens in clusters 6 and 9; some Boroughs in London and Newyork were grouped together.

- Cluster 6 contains locations where Coffee shops are the most common venues
- Cluster 9 contains locations where Cafes are the most common venues

Paris isn't clustered with any other city.